In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.21.0.4  222.2 KiB   16      100.0%            739395b2-9b2a-4828-a1a2-a85a7e358b24  rack1
UN  172.21.0.2  249.78 KiB  16      100.0%            d9d860d3-d1f1-4282-85f4-efd30b5b1258  rack1
UN  172.21.0.3  207.45 KiB  16      100.0%            b6057742-477a-4263-8869-1dc8926cb85e  rack1



In [2]:
#Connect to the Cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
#q1
#drop a weather keyspace if it already exists
cass.execute("drop keyspace if exists weather;")
#create a weather keyspace with 3x replication
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
#inside weather, create a station_record type containing two ints: tmin and tmax
cass.execute("""
CREATE TYPE
weather.station_record (
    tmin int,
    tmax int
)
""")
#inside weather, create a stations table
cass.execute("""
CREATE TABLE weather.stations (
    id text,
    name TEXT STATIC,
    date date,
    record station_record,
    state TEXT,
    PRIMARY KEY (id,date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

print(cass.execute("describe table weather.stations ").one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    state text,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [4]:
#Create a local Spark session
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa0ecdf7-d8c6-4efc-88f4-a25e5603acf7;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [5]:
sc = spark.sparkContext # for interacting directly with RDDs

In [6]:
#q2

In [7]:
df = spark.read.text("ghcnd-stations.txt")

In [8]:
pandas_df=df.toPandas()

In [9]:
pandas_df["station"] = pandas_df["value"].str[:11]

In [10]:
from pyspark.sql.functions import col, expr

In [11]:
df2 = df.withColumn("station", expr("substring(value, 0, 11)")).withColumn(
     "STATE",expr("substring(value,38,40)")).withColumn(
     "NAME",expr("substring(value, 41,71)"))
df2
df2=df2.filter(col("STATE").like("% WI %")).filter(col("station").like("%US%"))
df2.toPandas()

,value,station,STATE,NAME
0,US1WIAD0002 43.9544 -89.8096 294.4 WI ADAMS...,US1WIAD0002,WI ADAMS 0.4 E,ADAMS 0.4 E
1,US1WIAD0005 44.2053 -89.8480 305.7 WI NEKOO...,US1WIAD0005,WI NEKOOSA 8.0 SSE,NEKOOSA 8.0 SSE
2,US1WIAD0006 43.8858 -89.7259 307.8 WI GRAND...,US1WIAD0006,WI GRAND MARSH 1.0 W,GRAND MARSH 1.0 W
3,US1WIAD0008 43.8611 -89.7163 310.0 WI GRAND...,US1WIAD0008,WI GRAND MARSH 1.9 SSW,GRAND MARSH 1.9 SSW
4,US1WIAD0010 43.7864 -89.6417 293.8 WI OXFOR...,US1WIAD0010,WI OXFORD 4.0 W,OXFORD 4.0 W
...,...,...,...,...
1308,USW00094930 43.9333 -90.2667 280.1 WI VOLK ...,USW00094930,WI VOLK FLD ANG,VOLK FLD ANG
1309,USW00094940 43.9667 -90.7333 252.7 WI SPART...,USW00094940,WI SPARTA FT MCCOY,SPARTA FT MCCOY
1310,USW00094973 46.0303 -91.4425 368.8 WI HAYWA...,USW00094973,WI HAYWARD MUNI AP,HAYWARD MUNI AP
1311,USW00094985 44.6378 -90.1875 381.6 WI MARSH...,USW00094985,WI MARSHFIELD MUNI AP,MARSHFIELD MUNI AP


In [13]:
#Filter your results to the state of Wisconsin, collect the rows in your notebook so you can loop over them,
#and do an INSERT into your weather.stations table for each station ID and name.

# i=0
# for row in df2.collect():
#     station_WI = row["station"]
#     name_WI = row["NAME"]
    
#     # print(i)
#     # i+=1
#     # if(i==684 or i==685):
#     #     print(station_WI)
#     #     print(name_WI)
#     if "'" in name_WI or "'" in station_WI:
#         insert_query = f'INSERT INTO weather.stations (id, name) VALUES ("{station_WI}", "{name_WI}")'
#     else:
#         insert_query = f"INSERT INTO weather.stations (id, name) VALUES ('{station_WI}', '{name_WI}')"
#     cass.execute(insert_query)
insert_query = "INSERT INTO weather.stations (id, name) VALUES (?, ?)"
prepared_statement = cass.prepare(insert_query)
i=0
for row in df2.collect():
    station_WI = row["station"]
    name_WI = row["NAME"]
    values = (station_WI, name_WI)
    cass.execute(prepared_statement, values)
    i+=1

print(i)

1313


In [14]:
import pandas as pd
pd.DataFrame(cass.execute(
    """
    SELECT * FROM weather.stations
    """
))
pd.DataFrame(cass.execute(
    """
    SELECT COUNT(*) FROM weather.stations
    """
))

,count
0,1313


In [18]:
pd.DataFrame(cass.execute(
    """
    SELECT name FROM weather.stations
    WHERE id = 'USW00014837'
    """
))

,name
0,MADISON DANE CO RGNL AP 72641
